LangChain + Chatgpt model - Tested (need api key)
LangChain + llama2 - working on (trying on locally)

In [1]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2 tiktoken

import openai
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import os

os.environ["OPENAI_API_KEY"] = ""

zsh:1: command not found: pip


/Users/jay/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: No module named 'langchain.document_loaders'; 'langchain' is not a package

In [ ]:
# !wget any docs from online
# 

## Preprocess a PDF file

In [ ]:
reader = PdfReader("example.pdf")

raw_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

raw_text[:1000]

FileNotFoundError: [Errno 2] No such file or directory: 'Demian.pdf'

In [ ]:
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

ValidationError: ignored

In [ ]:
summarize_document_chain.run(raw_text)

" Hermann Hesse's Demian follows the journey of a young man as he discovers his true identity and learns to accept himself. He meets Max Demian, who helps him to understand the importance of self-reliance and the power of his own ideas. Through Demian's influence, the protagonist learns to accept his place in the world and to embrace his own unique story. Along the way, he meets Pistorius, who teaches him about Abraxas, and Eva, Demian's mother, who encourages him to find his dream and remain faithful to it. When war is declared, Sinclair goes to the front and finds something akin to humanity beneath the surface of the war. In the end, he finds comfort in the knowledge that his own image now resembles his friend and leader."

### How much cost? (example data)

- text-davinci, 4 requests
- 123,121 prompt + 8,930 completion = 110,500 tokens
- $0.47

## Question Answering Part

running time: 

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="Example Question 1")

NameError: name 'qa_document_chain' is not defined

In [ ]:
qa_document_chain.run(
    input_document=raw_text,
    question="Example Question 2")

NameError: name 'qa_document_chain' is not defined

### GPT-4

running time:

example:

90,000 tokens total == $6

> $0.06 / 1K tokens

In [ ]:
model = ChatOpenAI(model="gpt-4")
qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
qa_document_chain.run(
    input_document=raw_text,
    question="SAMPLE")

NameError: name 'ChatOpenAI' is not defined

##  CSV search, Excel, Aggregation

In [ ]:
import pandas as pd

df = pd.read_csv("sample.csv")

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
from langchain.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

agent.run("Sample question: how many lines are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 891
Thought: I now know the final answer
Final Answer: There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'